In [ ]:
import sys; sys.path.append("../../../utility_scripts/")
import matplotlib
from dotplot_utils import *
import pandas as pd
import seaborn as sns
# make this notebook work better with Scanpy
import warnings; warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
sc.set_figure_params(transparent=True, dpi_save=400)
sc.settings.figdir = "plots_patient/"

---
## Read in key dataframe with sample information

In [ ]:
sample_key = pd.read_csv("../../resources/visium_sample_key.csv", index_col=0)

---
## Define pseudotime ordering by `LCM_ROI`

In [ ]:
muts = pd.read_csv("../exome_genome/resources/LCM_variants_per_sample.csv", index_col=0)
mapper = {
    "WD86055_ROI4":"WD86055_ROI1",
    "WD86055_ROI2":"WD86055_ROI2",
    "WD86055_ROI3":"WD86055_ROI3",
    "WD86055_ROI1":"WD86055_ROI4",
    "WD86056_ROI2":"WD86055_ROI5",
    "WD86056_ROI1":"WD86055_ROI6",
    "WD86056_ROI4":"WD86055_ROI7",
    "WD86056_ROI3":"WD86055_ROI8",
    "WD33475_ROI4":"WD33475_ROI3",
    "WD33475_ROI5":"WD33475_ROI4",
}
muts.Tumor_Sample_Barcode.replace(mapper, inplace=True)
muts["Tumor_Sample_Barcode"] = [sample_key.loc[sample_key.block_name==x.split("_")[0],"patient_name"][0] + "_" + x.split("_")[1] for x in muts["Tumor_Sample_Barcode"]]
muts[["Patient", "ROI"]] = muts["Tumor_Sample_Barcode"].str.split("_", expand=True)

In [ ]:
muts = muts.merge(sample_key.loc[sample_key.tumor_type.isin(["MSS","MSI-H"]),["patient_name","tumor_type"]], left_on="Patient", right_on="patient_name", how="left").drop_duplicates()
muts = muts.drop(columns=["patient_name"])

In [ ]:
muts

In [ ]:
pt_orders = {}

muts_MSS = muts.loc[muts.tumor_type=="MSS",:].copy()
muts_MSS.Patient = muts_MSS.Patient.astype("category")
for pat in muts_MSS.Patient.cat.categories:
    pt_orders[pat] = list(muts.loc[muts.Patient==pat, "ROI"])[::-1]
    
muts_MSI = muts.loc[muts.tumor_type=="MSI-H",:].copy()
muts_MSI.Patient = muts_MSI.Patient.astype("category")
for pat in muts_MSI.Patient.cat.categories:
    pt_orders[pat] = list(muts.loc[muts.Patient==pat, "ROI"])[::-1]

In [ ]:
muts2 = pd.read_csv("../exome_genome/resources/pseudobulk_variants_per_sample.csv", index_col=0)

---

In [ ]:
outs = []
for s in sample_key.index:
    a = sc.read("datasets/{}_master.h5ad".format(s))
    print("Read adata from datasets/{}_master.h5ad".format(s))
    
    if "Variants" in a.obs.columns:
        a.obs.drop(columns="Variants", inplace=True)
    
    try:
        # add number of detected Variants
        a.obs = a.obs.reset_index().merge(muts[["Patient","ROI","Variants"]], left_on=["Patient","LCM_ROI"], right_on=["Patient","ROI"], how="left").drop(columns="ROI").set_index("index")
    except:
        print("No LCM variants found")
        try:
            a.obs["Variants"] = muts2.loc[muts2.Tumor_Sample_Barcode==sample_key.loc[s,:].block_name,"Variants"].values[0]
        except:
            print("No bulk variants found")


    # compile training data
    outs.append(a)

assert len(outs) == len(sample_key), "Check length of outs"

In [ ]:
# concatenate anndata objects
a_comb = outs[0].concatenate(
    outs[1:],
    join="outer",
    batch_categories=list(sample_key.index),
    fill_value=0,
)
del a_comb.var
del a_comb.obsm
a_comb  # view object

#### Adjust 'Variants' to 'TMB' for plotting

In [ ]:
a_comb.obs.rename(columns={"Variants":"TMB"}, inplace=True)

---
## Define gene signatures

In [ ]:
# recombine signatures into useful categories
signatures_epi_stroma = [
    'IFN stimulated EPI',
    'iCMS2',
    'iCMS3',
    'Stem cell index', # Gil Vasquez, et al. 2022
    'Basal',
    'Squamous',
    'Mesenchymal',
    'Glandular',
    'Ciliated',
    'Alveolar',
    'AC', # astrocyte
    'OPC', # oligodendrocyte progenitor
    'NPC', # neural progenitor
    'IES', # custom epithelial-intrinsic exclusion signature
]

signatures_myeloid = [
    'Myeloid',
    'cDC2',
    'cDC1',
    'Macrophage',
    'Classical monocytes',
    'M1',
    'M2',
    'Neutrophils',
    'Costimulatory MYE',
    'Stimulatory DCs',
    'IFN stimulated MYE',
]

signatures_lymphoid = [
    'T cell',
    'T reg',
    'T cell CD4',
    'T cell CD8',
    'T reg resting',
    'T reg suppressive',
    'T reg tissue homing',
    'T reg cytokines',
    'T reg activation',
    'TH1',
    'TH2',
    'TH17',
    'T cell exhaustion',
    'TRM', # tissue-resident memory T cells
    'NK cell',
    'B cell',
    'Plasma cell',
]

signatures_activity = [
    'EMT',
    'pEMT',
    'Senescence',
    'SASP',
    'Interferon',
    'Hypoxia',
    'Oxphos',
    'Stress',
    'Stress response',
    'Bacterial response',
    'Fibrosis',
    'Cytotoxicity',
    'Exhaustion',
    'Cytokines',
    'Chemokines',
    'MHC',
    'Fetal',
    'Stem',
    'Metaplasia',
    'Proliferation',
    'Translation',
    'Cycle',
    'Metal',
    'CytoTRACE',
    'CNV score',
]

signatures_curated = [
    # activity
    'pEMT',
    'Metaplasia',
    'Stem',
    'CytoTRACE',
    'IES',
    'CNV score',
    'MHC',
    'Fibrosis',
    'Bacterial response',
    # epi/stroma
    'IFN stimulated EPI',
    'iCMS2',
    'iCMS3',
    'Stem cell index', # Gil Vasquez, et al. 2022
    'Squamous',
    # lymphoid
    'T reg suppressive',
    'T cell CD4',
    'T cell CD8',
    'T cell exhaustion',
    # myeloid
    'IFN stimulated MYE',
    'Neutrophils',
]

signatures_fig_1 = [
    "Stem",
    "T cell CD4",
    "iCMS2",
    "iCMS3",
    "Metaplasia",
    "CytoTRACE",
    "CNV score",
    "T cell CD8"
]

signatures_fig_2 = [
    "Stem",
    "T cell CD4",
    "iCMS2",
    "iCMS3",
    "Metaplasia",
    "CytoTRACE",
    "CNV score",
    "T cell CD8",
    "IFN stimulated MYE",
    "T reg suppressive",
    "TMB",
    "T cell exhaustion",
]

---
## Rename things and set up for plotting

In [ ]:
# rename obs columns with MxIF stains in them
old_mxif = list(a_comb.obs.columns[a_comb.obs.columns.str.endswith("_MxIF")])
new_mxif = list(a_comb.obs.columns[a_comb.obs.columns.str.endswith("_MxIF")].str.split("_").str[0])
a_comb.obs.rename(columns=dict(zip(old_mxif,new_mxif)), inplace=True)

In [ ]:
# rename cell state columns in master anndata.obs
et = list(a_comb.obs.columns[a_comb.obs.columns.str.endswith("_VUMCrefNMF30")])
et_new = [x.replace("_VUMCrefNMF30", "") for x in et]
a_comb.obs.rename(columns=dict(zip(et, et_new)), inplace=True)

In [ ]:
a_comb

In [ ]:
cmap_dict = {
    # Tumor Type
    'SSL/HP':"#c4a4e1",'MSI-H':"#7a4fa3",'MSS':"#ffc101",'TA/TVA':"#fee799",'NL':"#1f77b4",
    # Tumor Location
    "Cecum":"#1f4e79","Ascending":"#2e74b7","Hepatic Flexure":"#bdd6ef","Transverse":"#ff717a","Descending":"#fe0001","Sigmoid":"#c00101",
    # this one's global
    "nan":"#ffffff",
    # These are black and white for T and F
    "T":"#000000","F":"#ffffff",
}
stage_colordict = dict(zip(["AD","I","II","III/IV"], sns.color_palette("Reds", len(["AD","I","II","III/IV"])).as_hex()))
grade_colordict = dict(zip(["G1","G2","G3"], sns.color_palette("Reds", len(["G1","G2","G3"])).as_hex()))
cmap_dict = {**cmap_dict, **stage_colordict, **grade_colordict}

In [ ]:
patient_colordict = dict(zip(sample_key.patient_name, [cmap_dict[x] for x in sample_key.tumor_type]))

In [ ]:
cmap = matplotlib.cm.get_cmap("plasma", 8)
mw_colordict = dict(zip(["D0","D1","D2","D3","D4","D5","D6","D7"], [matplotlib.colors.rgb2hex(cmap(i)) for i in range(cmap.N)]))

In [ ]:
signatures_activity_height = 7
signatures_epi_stroma_height = 4.1
signatures_myeloid_height = 3
signatures_lymphoid_height = 4.7
signatures_curated_height = 6
signatures_fig_1_height = 3
signatures_fig_2_height = 3.2
refNMF_height = 8
archetypes_height = 2.7
mxif_height = 8.2

In [ ]:
# define heatmap widths
milwrm_width = 4.1
tumor_type_width = 3.2
tumor_loc_width = 3.7
tumor_stage_width = 3.2
tumor_grade_width = 3.0
patient_width = 8.7
pathology_width = 4.0
evolution_width = 3.0
evolution_grouped_width = 2.5
cnv_domain_width = 3.0

---
# Overview plots

Summaries of global signatures, genes, archetypes by CNV clusters

---
## VUMC refNMF30 by `CNV Clone`

In [ ]:
# split refNMF cell states by compartment
nmf_epi = [
 'STM',
 'CRC1',
 'CRC2',
 'CT',
 'SSC',
 'CRC3',
 'EE1',
 'GOB',
 'CRC4',
 'ABS',
 'TUF',
 'EE2',
]
nmf_epi.sort()

nmf_stroma = [
 'END1',
 'FIB1',
 'FIB2',
 'FIB3',
 'FIB4',
 'END2',
]
nmf_stroma.sort()

nmf_immune = [
 'BL1',
 'MYE1',
 'TL1',
 'MYE2',
 'MYE3',
 'PLA',
 'MYE4',
 'MAS',
 'MYE5',
 'TL2',
 'TL3',
 'BL2'
]
nmf_immune.sort()

marker_states_dict = {
    "PPT": ["CNV score","TMB"],
    "Epithelial": nmf_epi,
    "Stromal": nmf_stroma,
    "Immune": nmf_immune,
}

In [ ]:
for pat in a_comb.obs.Patient.unique():
    print(pat)
    tmp = a_comb[a_comb.obs.Patient == pat, :].copy()

    # reorder clones
    tmp.obs["CNV clone"] = tmp.obs["CNV clone"].astype("category")
    clone_order = list(tmp.obs["CNV clone"].cat.categories)
    pre_clones = []
    for cluster in ["E", "S"]:
        if cluster in clone_order:
            clone_order.remove(cluster)
            pre_clones.append(cluster)
    for cluster in ["E", "S"]:
        if cluster in pre_clones:
            clone_order.insert(0, cluster)
            
    # loading uns dict back in from .npy file
    if pat=="HTA11_08622_A":
        cnv_clone_colordict = np.load("infercnv/cnv_clone_colors_HTA11_08622.npy", allow_pickle='TRUE').item()
    elif pat=="HTA11_08622_B":
        cnv_clone_colordict = np.load("infercnv/cnv_clone_colors_HTA11_08622.npy", allow_pickle='TRUE').item()
    else:
        cnv_clone_colordict = np.load("infercnv/cnv_clone_colors_{}.npy".format(pat), allow_pickle='TRUE').item()
    
    cody_heatmap(
        tmp,
        groupby="CNV clone",
        groupby_colordict=cnv_clone_colordict,
        features=sum(marker_states_dict.values(), []),
        cluster_vars=False,
        groupby_order=clone_order,
        vars_dict=marker_states_dict,
        cluster_obs=False,
        figsize=(0.8*len(tmp.obs["CNV clone"].cat.categories) if len(tmp.obs["CNV clone"].cat.categories) > 3 else 2.6, refNMF_height),
        save="plots_patient/{}_cnv_clone_VUMCrefNMF30_dotplot.png".format(pat),
        dpi=400,
        cmap="Greys",
        size_title="Fraction of spots\nin group (%)",
    )

---
## Archetypes by `CNV Clone`

In [ ]:
# split refNMF cell states by compartment
at_epi = [
 'A3_Endocrine',
 'A4_MUC',
 'A5_ABS',
 'A8_Stem',
]
at_epi.sort()

at_stroma = [
 'A6_SM',
]
at_stroma.sort()

at_immune = [
 'A1_Innate',
 'A2_Cytotoxic',
 'A7_Adaptive',
]
at_immune.sort()

archetypes_dict = {
    "PPT": ["CNV score","TMB"],
    "Epi.": at_epi,
    "Non-Epi.": at_stroma + at_immune,
}

In [ ]:
for pat in a_comb.obs.Patient.unique():
    print(pat)
    tmp = a_comb[a_comb.obs.Patient == pat, :].copy()

    # reorder clones
    tmp.obs["CNV clone"] = tmp.obs["CNV clone"].astype("category")
    clone_order = list(tmp.obs["CNV clone"].cat.categories)
    pre_clones = []
    for cluster in ["E", "S"]:
        if cluster in clone_order:
            clone_order.remove(cluster)
            pre_clones.append(cluster)
    for cluster in ["E", "S"]:
        if cluster in pre_clones:
            clone_order.insert(0, cluster)
            
    # loading uns dict back in from .npy file
    if pat=="HTA11_08622_A":
        cnv_clone_colordict = np.load("infercnv/cnv_clone_colors_HTA11_08622.npy", allow_pickle='TRUE').item()
    elif pat=="HTA11_08622_B":
        cnv_clone_colordict = np.load("infercnv/cnv_clone_colors_HTA11_08622.npy", allow_pickle='TRUE').item()
    else:
        cnv_clone_colordict = np.load("infercnv/cnv_clone_colors_{}.npy".format(pat), allow_pickle='TRUE').item()
    
    cody_heatmap(
        tmp,
        groupby="CNV clone",
        groupby_colordict=cnv_clone_colordict,
        features=sum(archetypes_dict.values(), []),
        cluster_vars=False,
        groupby_order=clone_order,
        vars_dict=archetypes_dict,
        cluster_obs=False,
        figsize=(0.8*len(tmp.obs["CNV clone"].cat.categories) if len(tmp.obs["CNV clone"].cat.categories) > 3 else 2.6, archetypes_height),
        save="plots_patient/{}_cnv_clone_archetypes_dotplot.png".format(pat),
        dpi=400,
        cmap="Greys",
        size_title="Fraction of spots\nin group (%)",
    )

---
## Activity Signatures by `CNV Clone`

In [ ]:
for pat in a_comb.obs.Patient.unique():
    print(pat)
    tmp = a_comb[a_comb.obs.Patient == pat, :].copy()

    # reorder clones
    tmp.obs["CNV clone"] = tmp.obs["CNV clone"].astype("category")
    clone_order = list(tmp.obs["CNV clone"].cat.categories)
    pre_clones = []
    for cluster in ["E", "S"]:
        if cluster in clone_order:
            clone_order.remove(cluster)
            pre_clones.append(cluster)
    for cluster in ["E", "S"]:
        if cluster in pre_clones:
            clone_order.insert(0, cluster)
    
    # loading uns dict back in from .npy file
    if pat=="HTA11_08622_A":
        cnv_clone_colordict = np.load("infercnv/cnv_clone_colors_HTA11_08622.npy", allow_pickle='TRUE').item()
    elif pat=="HTA11_08622_B":
        cnv_clone_colordict = np.load("infercnv/cnv_clone_colors_HTA11_08622.npy", allow_pickle='TRUE').item()
    else:
        cnv_clone_colordict = np.load("infercnv/cnv_clone_colors_{}.npy".format(pat), allow_pickle='TRUE').item()
    
    cody_heatmap(
        tmp,
        groupby="CNV clone",
        groupby_colordict=cnv_clone_colordict,
        features=signatures_activity,
        cluster_vars=True,
        groupby_order=clone_order,
        cluster_obs=False,
        figsize=(0.8*len(tmp.obs["CNV clone"].cat.categories) if len(tmp.obs["CNV clone"].cat.categories) > 3 else 2.6, signatures_activity_height),
        save="plots_patient/{}_cnv_clone_signatures_activity_dotplot.png".format(pat),
        dpi=400,
        cmap="Greys",
        size_title="Fraction of spots\nin group (%)",
    )

---
## Epithelial & Stromal Signatures by `CNV Clone`

In [ ]:
for pat in a_comb.obs.Patient.unique():
    print(pat)
    tmp = a_comb[a_comb.obs.Patient == pat, :].copy()

    # reorder clones
    tmp.obs["CNV clone"] = tmp.obs["CNV clone"].astype("category")
    clone_order = list(tmp.obs["CNV clone"].cat.categories)
    pre_clones = []
    for cluster in ["E", "S"]:
        if cluster in clone_order:
            clone_order.remove(cluster)
            pre_clones.append(cluster)
    for cluster in ["E", "S"]:
        if cluster in pre_clones:
            clone_order.insert(0, cluster)
    
    # loading uns dict back in from .npy file
    if pat=="HTA11_08622_A":
        cnv_clone_colordict = np.load("infercnv/cnv_clone_colors_HTA11_08622.npy", allow_pickle='TRUE').item()
    elif pat=="HTA11_08622_B":
        cnv_clone_colordict = np.load("infercnv/cnv_clone_colors_HTA11_08622.npy", allow_pickle='TRUE').item()
    else:
        cnv_clone_colordict = np.load("infercnv/cnv_clone_colors_{}.npy".format(pat), allow_pickle='TRUE').item()
    
    cody_heatmap(
        tmp,
        groupby="CNV clone",
        groupby_colordict=cnv_clone_colordict,
        features=signatures_epi_stroma,
        cluster_vars=True,
        groupby_order=clone_order,
        cluster_obs=False,
        figsize=(0.8*len(tmp.obs["CNV clone"].cat.categories) if len(tmp.obs["CNV clone"].cat.categories) > 3 else 2.6, signatures_epi_stroma_height),
        save="plots_patient/{}_cnv_clone_signatures_epi_stroma_dotplot.png".format(pat),
        dpi=400,
        cmap="Greys",
        size_title="Fraction of spots\nin group (%)",
    )

---
## Myeloid Signatures by `CNV Clone`

In [ ]:
for pat in a_comb.obs.Patient.unique():
    print(pat)
    tmp = a_comb[a_comb.obs.Patient == pat, :].copy()

    # reorder clones
    tmp.obs["CNV clone"] = tmp.obs["CNV clone"].astype("category")
    clone_order = list(tmp.obs["CNV clone"].cat.categories)
    pre_clones = []
    for cluster in ["E", "S"]:
        if cluster in clone_order:
            clone_order.remove(cluster)
            pre_clones.append(cluster)
    for cluster in ["E", "S"]:
        if cluster in pre_clones:
            clone_order.insert(0, cluster)
    
    # loading uns dict back in from .npy file
    if pat=="HTA11_08622_A":
        cnv_clone_colordict = np.load("infercnv/cnv_clone_colors_HTA11_08622.npy", allow_pickle='TRUE').item()
    elif pat=="HTA11_08622_B":
        cnv_clone_colordict = np.load("infercnv/cnv_clone_colors_HTA11_08622.npy", allow_pickle='TRUE').item()
    else:
        cnv_clone_colordict = np.load("infercnv/cnv_clone_colors_{}.npy".format(pat), allow_pickle='TRUE').item()
    
    cody_heatmap(
        tmp,
        groupby="CNV clone",
        groupby_colordict=cnv_clone_colordict,
        features=signatures_myeloid,
        cluster_vars=True,
        groupby_order=clone_order,
        cluster_obs=False,
        figsize=(0.8*len(tmp.obs["CNV clone"].cat.categories) if len(tmp.obs["CNV clone"].cat.categories) > 3 else 2.6, signatures_myeloid_height),
        save="plots_patient/{}_cnv_clone_signatures_myeloid_dotplot.png".format(pat),
        dpi=400,
        cmap="Greys",
        size_title="Fraction of spots\nin group (%)",
    )

---
## Lymphoid Signatures by `CNV Clone`

In [ ]:
for pat in a_comb.obs.Patient.unique():
    print(pat)
    tmp = a_comb[a_comb.obs.Patient == pat, :].copy()

    # reorder clones
    tmp.obs["CNV clone"] = tmp.obs["CNV clone"].astype("category")
    clone_order = list(tmp.obs["CNV clone"].cat.categories)
    pre_clones = []
    for cluster in ["E", "S"]:
        if cluster in clone_order:
            clone_order.remove(cluster)
            pre_clones.append(cluster)
    for cluster in ["E", "S"]:
        if cluster in pre_clones:
            clone_order.insert(0, cluster)
    
    # loading uns dict back in from .npy file
    if pat=="HTA11_08622_A":
        cnv_clone_colordict = np.load("infercnv/cnv_clone_colors_HTA11_08622.npy", allow_pickle='TRUE').item()
    elif pat=="HTA11_08622_B":
        cnv_clone_colordict = np.load("infercnv/cnv_clone_colors_HTA11_08622.npy", allow_pickle='TRUE').item()
    else:
        cnv_clone_colordict = np.load("infercnv/cnv_clone_colors_{}.npy".format(pat), allow_pickle='TRUE').item()
    
    cody_heatmap(
        tmp,
        groupby="CNV clone",
        groupby_colordict=cnv_clone_colordict,
        features=signatures_lymphoid,
        cluster_vars=True,
        groupby_order=clone_order,
        cluster_obs=False,
        figsize=(0.8*len(tmp.obs["CNV clone"].cat.categories) if len(tmp.obs["CNV clone"].cat.categories) > 3 else 2.6, signatures_lymphoid_height),
        save="plots_patient/{}_cnv_clone_signatures_lymphoid_dotplot.png".format(pat),
        dpi=400,
        cmap="Greys",
        size_title="Fraction of spots\nin group (%)",
    )

---
## Curated Signatures by `CNV Clone`

In [ ]:
for pat in a_comb.obs.Patient.unique():
    print(pat)
    tmp = a_comb[a_comb.obs.Patient == pat, :].copy()

    # reorder clones
    tmp.obs["CNV clone"] = tmp.obs["CNV clone"].astype("category")
    clone_order = list(tmp.obs["CNV clone"].cat.categories)
    pre_clones = []
    for cluster in ["E", "S"]:
        if cluster in clone_order:
            clone_order.remove(cluster)
            pre_clones.append(cluster)
    for cluster in ["E", "S"]:
        if cluster in pre_clones:
            clone_order.insert(0, cluster)
    
    # loading uns dict back in from .npy file
    if pat=="HTA11_08622_A":
        cnv_clone_colordict = np.load("infercnv/cnv_clone_colors_HTA11_08622.npy", allow_pickle='TRUE').item()
    elif pat=="HTA11_08622_B":
        cnv_clone_colordict = np.load("infercnv/cnv_clone_colors_HTA11_08622.npy", allow_pickle='TRUE').item()
    else:
        cnv_clone_colordict = np.load("infercnv/cnv_clone_colors_{}.npy".format(pat), allow_pickle='TRUE').item()
    
    cody_heatmap(
        tmp,
        groupby="CNV clone",
        groupby_colordict=cnv_clone_colordict,
        features=signatures_curated,
        cluster_vars=True,
        groupby_order=clone_order,
        cluster_obs=False,
        figsize=(0.8*len(tmp.obs["CNV clone"].cat.categories) if len(tmp.obs["CNV clone"].cat.categories) > 3 else 2.6, signatures_curated_height),
        save="plots_patient/{}_cnv_clone_signatures_curated_dotplot.png".format(pat),
        dpi=400,
        cmap="Greys",
        size_title="Fraction of spots\nin group (%)",
    )

---
## `signatures_fig_1_2` by `CNV Clone`

In [ ]:
for pat in a_comb.obs.Patient.unique():
    print(pat)
    tmp = a_comb[a_comb.obs.Patient == pat, :].copy()

    # reorder clones
    tmp.obs["CNV clone"] = tmp.obs["CNV clone"].astype("category")
    clone_order = list(tmp.obs["CNV clone"].cat.categories)
    pre_clones = []
    for cluster in ["E", "S"]:
        if cluster in clone_order:
            clone_order.remove(cluster)
            pre_clones.append(cluster)
    for cluster in ["E", "S"]:
        if cluster in pre_clones:
            clone_order.insert(0, cluster)
    
    # loading uns dict back in from .npy file
    if pat=="HTA11_08622_A":
        cnv_clone_colordict = np.load("infercnv/cnv_clone_colors_HTA11_08622.npy", allow_pickle='TRUE').item()
    elif pat=="HTA11_08622_B":
        cnv_clone_colordict = np.load("infercnv/cnv_clone_colors_HTA11_08622.npy", allow_pickle='TRUE').item()
    else:
        cnv_clone_colordict = np.load("infercnv/cnv_clone_colors_{}.npy".format(pat), allow_pickle='TRUE').item()
    
    cody_heatmap(
        tmp,
        groupby="CNV clone",
        groupby_colordict=cnv_clone_colordict,
        features=signatures_fig_1,
        cluster_vars=True,
        groupby_order=clone_order,
        cluster_obs=False,
        figsize=(0.8*len(tmp.obs["CNV clone"].cat.categories) if len(tmp.obs["CNV clone"].cat.categories) > 3 else 2.6, signatures_fig_1_height),
        save="plots_patient/{}_cnv_clone_signatures_fig_1_dotplot.png".format(pat),
        dpi=400,
        cmap="Greys",
        size_title="Fraction of spots\nin group (%)",
    )

In [ ]:
for pat in a_comb.obs.Patient.unique():
    print(pat)
    tmp = a_comb[a_comb.obs.Patient == pat, :].copy()

    # reorder clones
    tmp.obs["CNV clone"] = tmp.obs["CNV clone"].astype("category")
    clone_order = list(tmp.obs["CNV clone"].cat.categories)
    pre_clones = []
    for cluster in ["E", "S"]:
        if cluster in clone_order:
            clone_order.remove(cluster)
            pre_clones.append(cluster)
    for cluster in ["E", "S"]:
        if cluster in pre_clones:
            clone_order.insert(0, cluster)
    
    # loading uns dict back in from .npy file
    if pat=="HTA11_08622_A":
        cnv_clone_colordict = np.load("infercnv/cnv_clone_colors_HTA11_08622.npy", allow_pickle='TRUE').item()
    elif pat=="HTA11_08622_B":
        cnv_clone_colordict = np.load("infercnv/cnv_clone_colors_HTA11_08622.npy", allow_pickle='TRUE').item()
    else:
        cnv_clone_colordict = np.load("infercnv/cnv_clone_colors_{}.npy".format(pat), allow_pickle='TRUE').item()
    
    cody_heatmap(
        tmp,
        groupby="CNV clone",
        groupby_colordict=cnv_clone_colordict,
        features=signatures_fig_2,
        cluster_vars=True,
        groupby_order=clone_order,
        cluster_obs=False,
        figsize=(0.8*len(tmp.obs["CNV clone"].cat.categories) if len(tmp.obs["CNV clone"].cat.categories) > 3 else 2.6, signatures_fig_2_height),
        save="plots_patient/{}_cnv_clone_signatures_fig_2_dotplot.png".format(pat),
        dpi=400,
        cmap="Greys",
        size_title="Fraction of spots\nin group (%)",
    )

---
## MxIF Intensities by `CNV Clone`

In [ ]:
for pat in a_comb.obs.Patient.unique():
    print(pat)
    tmp = a_comb[a_comb.obs.Patient == pat, :].copy()

    # reorder clones
    tmp.obs["CNV clone"] = tmp.obs["CNV clone"].astype("category")
    clone_order = list(tmp.obs["CNV clone"].cat.categories)
    pre_clones = []
    for cluster in ["E", "S"]:
        if cluster in clone_order:
            clone_order.remove(cluster)
            pre_clones.append(cluster)
    for cluster in ["E", "S"]:
        if cluster in pre_clones:
            clone_order.insert(0, cluster)
    
    # loading uns dict back in from .npy file
    if pat=="HTA11_08622_A":
        cnv_clone_colordict = np.load("infercnv/cnv_clone_colors_HTA11_08622.npy", allow_pickle='TRUE').item()
    elif pat=="HTA11_08622_B":
        cnv_clone_colordict = np.load("infercnv/cnv_clone_colors_HTA11_08622.npy", allow_pickle='TRUE').item()
    else:
        cnv_clone_colordict = np.load("infercnv/cnv_clone_colors_{}.npy".format(pat), allow_pickle='TRUE').item()
    
    cody_heatmap(
        tmp,
        groupby="CNV clone",
        groupby_colordict=cnv_clone_colordict,
        features=new_mxif,
        cluster_vars=True,
        groupby_order=clone_order,
        cluster_obs=False,
        figsize=(0.8*len(tmp.obs["CNV clone"].cat.categories) if len(tmp.obs["CNV clone"].cat.categories) > 3 else 2.6, mxif_height),
        save="plots_patient/{}_cnv_clone_MxIF_dotplot.png".format(pat),
        dpi=400,
        cmap="Greys",
        size_title="Fraction of spots\nin group (%)",
    )

---
## Immune Exclusion Markers by `CNV Clone`

In [ ]:
custom_dict = {
    "PPT": ["CNV score","TMB"],
    "Excl.": ["IES","DDR1","PAK4","TGFBI","DPEP1","Fibrosis"],
    "uEnv.": ["FIB2","FIB3","MYE2"],
    "Inf.": ["T cell CD4","T reg suppressive","TL1","T cell CD8","TL2","TL3","MYE4","MYE5"],
    "Act.": ["Oxphos","Hypoxia","pEMT"],
    "MSS": ["iCMS2","Stem","CRC2"],
    "MSI": ["iCMS3","SSC","Metaplasia"],
    "NL": ["GOB","ABS","CT"],
}
custom_dict_height = 7.4

In [ ]:
for pat in a_comb.obs.Patient.unique():
    print(pat)
    tmp = a_comb[a_comb.obs.Patient == pat, :].copy()

    # reorder clones
    tmp.obs["CNV clone"] = tmp.obs["CNV clone"].astype("category")
    clone_order = list(tmp.obs["CNV clone"].cat.categories)
    pre_clones = []
    for cluster in ["E", "S"]:
        if cluster in clone_order:
            clone_order.remove(cluster)
            pre_clones.append(cluster)
    for cluster in ["E", "S"]:
        if cluster in pre_clones:
            clone_order.insert(0, cluster)
            
    tmp.obs.drop(columns=["DPEP1"],inplace=True)
    
    # loading uns dict back in from .npy file
    if pat=="HTA11_08622_A":
        cnv_clone_colordict = np.load("infercnv/cnv_clone_colors_HTA11_08622.npy", allow_pickle='TRUE').item()
    elif pat=="HTA11_08622_B":
        cnv_clone_colordict = np.load("infercnv/cnv_clone_colors_HTA11_08622.npy", allow_pickle='TRUE').item()
    else:
        cnv_clone_colordict = np.load("infercnv/cnv_clone_colors_{}.npy".format(pat), allow_pickle='TRUE').item()
    
    cody_heatmap(
        tmp,
        groupby="CNV clone",
        groupby_colordict=cnv_clone_colordict,
        features=sum(custom_dict.values(), []),
        cluster_vars=False,
        groupby_order=clone_order,
        vars_dict=custom_dict,
        cluster_obs=False,
        figsize=(0.8*len(tmp.obs["CNV clone"].cat.categories) if len(tmp.obs["CNV clone"].cat.categories) > 3 else 2.6, custom_dict_height),
        save="plots_patient/{}_cnv_immexcl_dotplot.png".format(pat),
        dpi=400,
        cmap="Greys",
        size_title="Fraction of spots\nin group (%)",
    )

---
# Global plots across all patients

In [ ]:
a_comb.obs["CNV clone"] = a_comb.obs.Patient + " " + a_comb.obs["CNV Clone"]

---
# Curated ordering

In [ ]:
sample_key.loc[sample_key.tumor_type=="SSL/HP","CIN Status"] = "HM"

In [ ]:
CIN_PATs = sample_key.loc[sample_key["CIN Status"].fillna("CIN+")=="CIN+","patient_name"].unique()

In [ ]:
CIN_PATs

In [ ]:
HM_PATs = sample_key.loc[sample_key["CIN Status"].fillna("CIN+")=="HM","patient_name"].unique()

In [ ]:
HM_PATs

In [ ]:
tmp1_adata = a_comb[a_comb.obs.Patient.isin(CIN_PATs), :].copy()
tmp1 = tmp1_adata.obs.groupby(["CNV clone"]).mean().sort_values(["CNV score"], ascending=[True]).reset_index()

tmp2_adata = a_comb[a_comb.obs.Patient.isin(HM_PATs), :].copy()
tmp2 = tmp2_adata.obs.groupby(["CNV clone"]).mean().sort_values(["TMB"], ascending=[True]).reset_index()

In [ ]:
tmp1_order = list(tmp1.loc[~(tmp1["CNV clone"].str.endswith(" S") | tmp1["CNV clone"].str.endswith(" E")), "CNV clone"])
tmp2_order = list(tmp2.loc[~(tmp2["CNV clone"].str.endswith(" S") | tmp2["CNV clone"].str.endswith(" E")), "CNV clone"])
custom_order = tmp1_order + tmp2_order

In [ ]:
patient_colordict = dict(zip(sample_key.patient_name, [cmap_dict[x] for x in sample_key.tumor_type]))
tmp1_colors = dict(zip(tmp1_order, [patient_colordict[x.split(" ")[0]] for x in tmp1_order]))
tmp2_colors = dict(zip(tmp2_order, [patient_colordict[x.split(" ")[0]] for x in tmp2_order]))
custom_colors = dict(zip(custom_order, [patient_colordict[x.split(" ")[0]] for x in custom_order]))

In [ ]:
cody_heatmap(
    tmp1_adata[tmp1_adata.obs["CNV clone"].isin(tmp1_order), :],
    groupby="CNV clone",
    features=sum(marker_states_dict.values(), []),
    cluster_vars=False,
    groupby_order=tmp1_order,
    groupby_colordict=tmp1_colors,
    vars_dict=marker_states_dict,
    cluster_obs=False,
    figsize=(9.2, refNMF_height),
    save="plots_patient/patientCNVclone_CIN+_ordercustom_refNMF30_dotplot.png",
    dpi=400,
    cmap="Greys",
    size_title="Fraction of spots\nin group (%)",
)

In [ ]:
cody_heatmap(
    tmp2_adata[tmp2_adata.obs["CNV clone"].isin(tmp2_order), :],
    groupby="CNV clone",
    features=sum(marker_states_dict.values(), []),
    cluster_vars=False,
    groupby_order=tmp2_order,
    groupby_colordict=tmp2_colors,
    vars_dict=marker_states_dict,
    cluster_obs=False,
    figsize=(6.2, refNMF_height),
    save="plots_patient/patientCNVclone_HM_ordercustom_refNMF30_dotplot.png",
    dpi=400,
    cmap="Greys",
    size_title="Fraction of spots\nin group (%)",
)

In [ ]:
cody_heatmap(
    a_comb[a_comb.obs["CNV clone"].isin(custom_order), :],
    groupby="CNV clone",
    features=sum(marker_states_dict.values(), []),
    cluster_vars=False,
    groupby_order=custom_order,
    groupby_colordict=custom_colors,
    vars_dict=marker_states_dict,
    cluster_obs=False,
    figsize=(13.2, refNMF_height),
    save="plots_patient/patientCNVclone_ordercustom_refNMF30_dotplot.png",
    dpi=400,
    cmap="Greys",
    size_title="Fraction of spots\nin group (%)",
)

In [ ]:
p = cody_heatmap(
    a_comb[a_comb.obs["CNV clone"].isin(custom_order), :],
    groupby="CNV Clone",
    features=sum(archetypes_dict.values(), []),
    cluster_vars=False,
    groupby_order=custom_order,
    groupby_colordict=custom_colors,
    vars_dict=archetypes_dict,
    cluster_obs=False,
    figsize=(13.2, archetypes_height),
    save="plots_patient/patientCNVclone_ordercustom_archetypes_dotplot.png",
    dpi=400,
    cmap="Greys",
    size_title="Fraction of spots\nin group (%)",
)

In [ ]:
cody_heatmap(
    a_comb[a_comb.obs["CNV clone"].isin(custom_order), :],
    groupby="CNV clone",
    features=signatures_activity,
    cluster_vars=True,
    groupby_order=custom_order,
    groupby_colordict=custom_colors,
    vars_dict=None,
    cluster_obs=False,
    figsize=(13.2, signatures_activity_height),
    save="plots_patient/patientCNVclone_ordercustom_signatures_activity_dotplot.png",
    dpi=400,
    cmap="Greys",
    size_title="Fraction of spots\nin group (%)",
)

In [ ]:
cody_heatmap(
    a_comb[a_comb.obs["CNV clone"].isin(custom_order), :],
    groupby="CNV clone",
    features=signatures_epi_stroma + ["TMB","CNV score"],
    cluster_vars=True,
    groupby_order=custom_order,
    groupby_colordict=custom_colors,
    vars_dict=None,
    cluster_obs=False,
    figsize=(13.2, signatures_epi_stroma_height),
    save="plots_patient/patientCNVclone_ordercustom_signatures_epi_stroma_dotplot.png",
    dpi=400,
    cmap="Greys",
    size_title="Fraction of spots\nin group (%)",
)

In [ ]:
cody_heatmap(
    a_comb[a_comb.obs["CNV clone"].isin(custom_order), :],
    groupby="CNV clone",
    features=signatures_myeloid + ["TMB","CNV score"],
    cluster_vars=True,
    groupby_order=custom_order,
    groupby_colordict=custom_colors,
    vars_dict=None,
    cluster_obs=False,
    figsize=(13.2, signatures_myeloid_height),
    save="plots_patient/patientCNVclone_ordercustom_signatures_myeloid_dotplot.png",
    dpi=400,
    cmap="Greys",
    size_title="Fraction of spots\nin group (%)",
)

In [ ]:
cody_heatmap(
    a_comb[a_comb.obs["CNV clone"].isin(custom_order), :],
    groupby="CNV clone",
    features=signatures_lymphoid + ["TMB","CNV score"],
    cluster_vars=True,
    groupby_order=custom_order,
    groupby_colordict=custom_colors,
    vars_dict=None,
    cluster_obs=False,
    figsize=(13.2, signatures_lymphoid_height),
    save="plots_patient/patientCNVclone_ordercustom_signatures_lymphoid_dotplot.png",
    dpi=400,
    cmap="Greys",
    size_title="Fraction of spots\nin group (%)",
)

In [ ]:
cody_heatmap(
    a_comb[a_comb.obs["CNV clone"].isin(custom_order), :],
    groupby="CNV clone",
    features=signatures_curated,
    cluster_vars=True,
    groupby_order=custom_order,
    groupby_colordict=custom_colors,
    vars_dict=None,
    cluster_obs=False,
    figsize=(13.2, signatures_curated_height),
    save="plots_patient/patientCNVclone_ordercustom_signatures_curated_dotplot.png",
    dpi=400,
    cmap="Greys",
    size_title="Fraction of spots\nin group (%)",
)

In [ ]:
cody_heatmap(
    a_comb[a_comb.obs["CNV clone"].isin(custom_order), :],
    groupby="CNV clone",
    features=new_mxif + ["TMB","CNV score"],
    cluster_vars=True,
    groupby_order=custom_order,
    groupby_colordict=custom_colors,
    vars_dict=None,
    cluster_obs=False,
    figsize=(13.2, mxif_height),
    save="plots_patient/patientCNVclone_ordercustom_MxIF_dotplot.png",
    dpi=400,
    cmap="Greys",
    size_title="Fraction of spots\nin group (%)",
)

---

In [ ]:
a_comb.obs.drop(columns=["DPEP1"],inplace=True)

In [ ]:
tmp1_adata.obs.drop(columns=["DPEP1"],inplace=True)

In [ ]:
tmp2_adata.obs.drop(columns=["DPEP1"],inplace=True)

In [ ]:
custom_dict = {
    "PT": ["CNV score","TMB"],
    "Excl.": ["IES","DDR1","PAK4","TGFBI","DPEP1","Fibrosis"],
    "uEnv.": ["FIB2","FIB3","MYE2"],
    "Inf.": ["T cell CD4","T reg suppressive","TL1","T cell CD8","TL2","TL3","MYE4","MYE5"],
    "Act.": ["Oxphos","Hypoxia","pEMT"],
    "MSS": ["iCMS2","Stem","CRC2"],
    "MSI": ["iCMS3","SSC","Metaplasia"],
    "NL": ["GOB","ABS","CT"],
}
custom_dict_height = 7.4

In [ ]:
cody_heatmap(
    tmp1_adata[tmp1_adata.obs["CNV clone"].isin(tmp1_order), :],
    groupby="CNV clone",
    features=sum(custom_dict.values(), []),
    cluster_vars=False,
    groupby_order=tmp1_order,
    groupby_colordict=tmp1_colors,
    vars_dict=custom_dict,
    cluster_obs=False,
    figsize=(9.2, custom_dict_height),
    save="plots_patient/patientCNVclone_CIN+_ordercustom_immexcl_dotplot.png",
    dpi=400,
    cmap="Greys",
    size_title="Fraction of spots\nin group (%)",
)

In [ ]:
cody_heatmap(
    tmp2_adata[tmp2_adata.obs["CNV clone"].isin(tmp2_order), :],
    groupby="CNV clone",
    features=sum(custom_dict.values(), []),
    cluster_vars=False,
    groupby_order=tmp2_order,
    groupby_colordict=tmp2_colors,
    vars_dict=custom_dict,
    cluster_obs=False,
    figsize=(6.2, custom_dict_height),
    save="plots_patient/patientCNVclone_HM_ordercustom_immexcl_dotplot.png",
    dpi=400,
    cmap="Greys",
    size_title="Fraction of spots\nin group (%)",
)

In [ ]:
cody_heatmap(
    a_comb[a_comb.obs["CNV clone"].isin(custom_order), :],
    groupby="CNV clone",
    features=sum(custom_dict.values(), []),
    cluster_vars=False,
    groupby_order=custom_order,
    groupby_colordict=custom_colors,
    vars_dict=custom_dict,
    cluster_obs=False,
    figsize=(13.5, custom_dict_height),
    save="plots_patient/patientCNVclone_ordercustom_immexcl_dotplot.png",
    dpi=400,
    cmap="Greys",
)

---

In [ ]:
tmp1 = tmp1.loc[~(tmp1["CNV clone"].str.endswith(" S") | tmp1["CNV clone"].str.endswith(" E")), :].copy()

In [ ]:
tmp2 = tmp2.loc[~(tmp2["CNV clone"].str.endswith(" S") | tmp2["CNV clone"].str.endswith(" E")), :].copy()

In [ ]:
list(tmp1.columns)

In [ ]:
import statsmodels.api as sm
import matplotlib.pyplot as plt

#create Q-Q plot with 45-degree line added to plot
fig = sm.qqplot(data, line='45')
plt.show()

In [ ]:
q = np.linspace(0, 100, 101)

In [ ]:
sns.scatterplot(np.percentile(tmp1["CNV score"], q), np.percentile(tmp1["iCMS2"], q))